In [15]:
import requests, zipfile, io


In [2]:
import geopandas as gpd

In [22]:
import pandas as pd
import json
import os
from sqlalchemy import create_engine
import sqlalchemy 
from shapely.geometry import Polygon
import geoalchemy2
from mapbox import Uploader
import sys

In [23]:
sys.path.append('..')

In [4]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#códigos ISO y coordenadas centroides

In [7]:
url='https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/master/utils/iso3166-2.csv'

In [8]:
iso=pd.read_csv(url)

In [9]:
name='iso'
schema='geo'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')

schema geo exists, will not be created


In [10]:
new_cols=[x.replace('(','').replace(')','') for x in iso.columns]

In [11]:
iso.columns=new_cols

In [12]:
iso

,Country Code,Country Name,Code,Subdivision Name Used,Subdivision Category,Number,Latitude,Longitude,Subdivision Name es,Subdivision Name qu,Subdivision Name ay,Subdivision Name ht,Subdivision Name fr,Parent Subdivision,Local Variant
0,AR,Argentina,AR-B,Buenos Aires,Province,NaN,-37.201728,-59.841070,Buenos Aires,NaN,NaN,NaN,NaN,NaN,NaN
1,AR,Argentina,AR-K,Catamarca,Province,NaN,-28.471588,-65.787721,Catamarca,NaN,NaN,NaN,NaN,NaN,NaN
2,AR,Argentina,AR-H,Chaco,Province,NaN,-27.425717,-59.024378,Chaco,NaN,NaN,NaN,NaN,NaN,NaN
3,AR,Argentina,AR-U,Chubut,Province,NaN,-43.293425,-65.111482,Chubut,NaN,NaN,NaN,NaN,NaN,NaN
4,AR,Argentina,AR-C,Ciudad Autónoma de Buenos Aires,City,NaN,-34.603684,-58.381559,Ciudad Autónoma de Buenos Aires,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,VE,Venezuela,VE-T,Trujillo,State,NaN,9.430253,-70.526493,Trujillo,NaN,NaN,NaN,NaN,NaN,NaN
405,VE,Venezuela,VE-S,Táchira,State,NaN,7.913700,-72.141613,Táchira,NaN,NaN,NaN,NaN,NaN,NaN
406,VE,Venezuela,VE-X,Vargas,State,NaN,10.589002,-66.736734,Vargas,NaN,NaN,NaN,NaN,NaN,NaN
407,VE,Venezuela,VE-U,Yaracuy,State,NaN,10.340192,-68.745306,Yaracuy,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
iso.to_sql(name, schema=schema,con=cnx,if_exists='replace')

In [14]:
### DATOS CENSO INE (MANZANA CENSAL)---->INCLUYE ZONAS CENSALES

URL='https://services5.arcgis.com/hUyD8u3TeZLKPe4T/arcgis/rest/services/Distrito_Densid_Superficie/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
zonas = requests.get(URL)
zonas=zonas.json()
crs=zonas['spatialReference']['wkid']
zonas = pd.io.json.json_normalize(zonas['features'])
#zonas=gpd.GeoDataFrame(crs="EPSG:"+str(crs), geometry=geometry.rings)
zonas['geometry']=[ Polygon(x[0]) for x in zonas['geometry.rings'] ]
zonas=zonas.drop('geometry.rings',axis=1)
zonas=gpd.GeoDataFrame(zonas,crs="EPSG:"+str(crs))
zonas=zonas.dropna(subset=['geometry'])

<ipython-input-14-3e847043e96f>:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  zonas = pd.io.json.json_normalize(zonas['features'])


In [30]:
service = Uploader(access_token="")

In [17]:
geojson=zonas.to_json()

In [27]:
zonas.to_file("../ETLcache/test.geojson", driver='GeoJSON')

In [31]:
with open("../ETLcache/test.geojson", 'rb') as src:
    upload_resp = service.upload(src, 'test_dataset')

In [32]:
upload_resp

<Response [201]>

In [33]:
upload_id = upload_resp.json()['id']

In [34]:
upload_id

'ckjf6eid90htp22lma7l8a3yb'

In [35]:
status_resp = service.status(upload_id).json()

In [ ]:
zonas['geometry'].astype(str)

In [69]:
zonas.to_postgis(name='zonas_censales', schema=schema,con=cnx,if_exists='replace')

ProgrammingError: (psycopg2.errors.UndefinedObject) type "geometry" does not exist
LINE 30:  geometry geometry(POLYGON,4326)
                   ^

[SQL: 
CREATE TABLE geo.zonas_censales (
	"attributes.OBJECTID" BIGINT, 
	"attributes.REGION" TEXT, 
	"attributes.NOM_REGION" TEXT, 
	"attributes.PROVINCIA" TEXT, 
	"attributes.NOM_PROVIN" TEXT, 
	"attributes.COMUNA" TEXT, 
	"attributes.NOM_COMUNA" TEXT, 
	"attributes.COD_DISTRI" BIGINT, 
	"attributes.TIPO_DISTR" TEXT, 
	"attributes.CODIGO_C17" TEXT, 
	"attributes.T_HOM_R" BIGINT, 
	"attributes.T_MUJ_R" BIGINT, 
	"attributes.T_POB_R" BIGINT, 
	"attributes.T_HOM_U" BIGINT, 
	"attributes.T_MUJ_U" BIGINT, 
	"attributes.T_POB_U" BIGINT, 
	"attributes.T_HOM" BIGINT, 
	"attributes.T_MUJ" BIGINT, 
	"attributes.T_POB" BIGINT, 
	"attributes.T_VIV_U" BIGINT, 
	"attributes.T_VIV_R" BIGINT, 
	"attributes.T_VIV" BIGINT, 
	"attributes.Shape_Leng" FLOAT(53), 
	"attributes.Shape__Area" FLOAT(53), 
	"attributes.Shape__Length" FLOAT(53), 
	"attributes.SUPERFI

In [14]:
#comunas
#url='https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10396/2/Comunas.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall("/data/ETLcache/comunas_shp")

In [15]:
#provincias
#url='https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10397/2/Provincias.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall("/data/ETLcache/provincias_shp")

In [16]:
#regiones
#url='https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10398/2/Regiones.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall("/data/ETLcache/regiones_shp")

In [17]:
#url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/utils/Mapa%20de%20Chile%20con%20Zoom%20en%20Santiago.json'
#response = requests.get(url)
#data = response.json()

In [18]:
#data

In [19]:
#url='https://gist.githubusercontent.com/rhernandog/7d055482f5cc803852a762de873bea62/raw/2bed9aed94ab644533b5e624a4e8f165a4650d48/regiones-provincias-comunas.json'
#response = requests.get(url)
#data = response.json()

In [20]:
#gdf = gpd.GeoDataFrame(data)

In [21]:
#from mapbox import Uploader

In [22]:
#conxn = Uploader(access_token='')

In [23]:
#resp = conxn.upload('RGB.byte.tif', 'RGB-byte-tif')

In [24]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/geo.ipynb')

0